<a href="https://colab.research.google.com/github/AlamShihab/AI-Agent-FAQ-Chatbot/blob/main/FAQ_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install -q openai langchain langchain-community langchain-core langdetect deep_translator flask flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import json
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langdetect import detect
from deep_translator import GoogleTranslator
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from google.colab import userdata

In [3]:
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('faqchatbot')  # Ensure you store your API key in Colab userdata

In [5]:
from google.colab import files

# Upload your CSV file
FAQ_FILE = files.upload()
# Load FAQs from a JSON file
#FAQ_FILE = "faq_data.json"

# Function to load FAQ data
def load_faq_data():
    try:
        with open(FAQ_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return ["Error! "]

# Function to save FAQ data
def save_faq_data(faq_data):
    with open(FAQ_FILE, "w") as file:
        json.dump(faq_data, file, indent=4)

faq_data = load_faq_data()

Saving faqchatbot.json to faqchatbot.json


TypeError: expected str, bytes or os.PathLike object, not dict

In [6]:
# Initialize the GPT-4o mini model
chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Create a template for chatbot responses
faq_template = PromptTemplate(
    input_variables=["question"],
    template="You are an AI chatbot. Answer this question based on the provided FAQ data:\n{faq_data}\nQuestion: {question}"
)

# Create a chain for handling chatbot interactions
faq_chain = LLMChain(
    prompt=faq_template,
    llm=chat_model
)

# Function to log user conversations
LOG_FILE = "chat_log.txt"

def log_conversation(user_input, response):
    with open(LOG_FILE, "a") as file:
        file.write(f"User: {user_input}\nBot: {response}\n\n")

# Function to add new FAQs
def add_faq():
    question = input("Enter the new FAQ question: ")
    answer = input("Enter the answer: ")

    # Load existing FAQs
    faq_data = load_faq_data()

    # Append new FAQ
    faq_data.append({"question": question, "answer": answer})

    # Save back to JSON file
    save_faq_data(faq_data)

    print("✅ New FAQ added successfully!")



<ipython-input-6-e39c268c188f>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
<ipython-input-6-e39c268c188f>:11: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  faq_chain = LLMChain(


In [8]:
# Function to handle user queries
def chatbot():
    print("🤖 Welcome to the AI FAQ Chatbot!")
    print("Type 'add' to contribute a new FAQ or 'exit' to stop.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("👋 Goodbye!")
            break
        elif user_input.lower() == 'add':
            add_faq()
        else:
            detected_lang = detect(user_input)
            response = faq_chain.run({"question": user_input, "faq_data": faq_data})

            if detected_lang != "en":
                translated_response = GoogleTranslator(source="en", target=detected_lang).translate(response)
                print(f"Bot ({detected_lang}):", translated_response)
                log_conversation(user_input, translated_response)
            else:
                print("Bot:", response)
                log_conversation(user_input, response)

chatbot()

🤖 Welcome to the AI FAQ Chatbot!
Type 'add' to contribute a new FAQ or 'exit' to stop.
You: ki khobor?
Bot (sl): Zdi se, da je prišlo do napake pri posredovanih informacijah. Bi lahko razjasnili ali navedli več podrobnosti o svojem vprašanju? Tukaj sem za pomoč!
You: kmn acho?
Bot (sk): Zdá sa, že poskytnuté údaje FAQ neobsahujú žiadne relevantné informácie na riešenie vašej otázky. Mohli by ste poskytnúť viac kontextu alebo objasniť, čo sa pýtate?
You: exit
👋 Goodbye!


In [9]:
#test 1
from langdetect import detect
from deep_translator import GoogleTranslator

# Function to handle user queries
def chatbot():
    print("🤖 Welcome to the AI FAQ Chatbot!")
    print("Type 'add' to contribute a new FAQ or 'exit' to stop.")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("👋 Goodbye!")
            break
        elif user_input.lower() == 'add':
            add_faq()
        else:
            # Avoid language misdetection for short inputs
            if len(user_input) < 3:
                detected_lang = "en"
            else:
                detected_lang = detect(user_input)

            response = faq_chain.run({"question": user_input, "faq_data": faq_data})

            if detected_lang != "en":
                translated_response = GoogleTranslator(source="en", target=detected_lang).translate(response)
                print(f"Bot ({detected_lang}):", translated_response)
                log_conversation(user_input, translated_response)
            else:
                print("Bot:", response)
                log_conversation(user_input, response)

chatbot()


🤖 Welcome to the AI FAQ Chatbot!
Type 'add' to contribute a new FAQ or 'exit' to stop.
You: ki khbor?
Bot (sl): Zdi se, da so navedeni podatki o pogostih vprašanjih omejeni in ne morem najti posebnih informacij, povezanih z vašim vprašanjem "Ki Khbor?" Če bi lahko navedli več konteksta ali razjasnili vaše vprašanje, bi z veseljem pomagal!
You: exit
👋 Goodbye!


In [12]:
#Test 2

import openai
from deep_translator import GoogleTranslator

# Set your OpenAI API key
openai.api_key = userdata.get('faqchatbot2')

# Function to detect if the input is Romanized Bangla
def is_romanized_bangla_with_llm(text):
    prompt = f"Is this text Romanized Bangla? Respond with 'Yes' or 'No' only. Text: {text}"

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    return "Yes" in response["choices"][0]["message"]["content"]

# Function to generate chatbot responses using LLM
def get_chatbot_response(text):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": text}]
    )
    return response["choices"][0]["message"]["content"]

# Main chatbot function
def chatbot():
    print("🤖 Welcome to the AI Chatbot!")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Detect if input is Romanized Bangla
        if is_romanized_bangla_with_llm(user_input):
            print("🟢 Detected Romanized Bangla.")

            # Get LLM response in English
            response = get_chatbot_response(user_input)

            # Translate to Bangla
            translated_response = GoogleTranslator(source="en", target="bn").translate(response)

            print(f"Bot (বাংলা): {translated_response}")

        else:
            response = get_chatbot_response(user_input)
            print(f"Bot: {response}")

chatbot()


🤖 Welcome to the AI Chatbot!
You: hi! ki khbr?


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [14]:
#test 2.1

#Test 2

import openai
from deep_translator import GoogleTranslator

# Set your OpenAI API key
openai.api_key = userdata.get('faqchatbot')

# Function to detect if the input is Romanized Bangla
def is_romanized_bangla_with_llm(text):
    prompt = f"Is this text Romanized Bangla? Respond with 'Yes' or 'No' only. Text: {text}"

    # Use openai.Completion.create instead of openai.ChatCompletion.create
    response = openai.Completion.create(
        engine="gpt-4o-mini",  # Replace 'model' with 'engine' for the new API
        prompt=prompt,
        max_tokens=10  # Limit the response length for a simple yes/no
    )

    return "Yes" in response.choices[0].text.strip()

# Function to generate chatbot responses using LLM
def get_chatbot_response(text):
    # Use openai.Completion.create instead of openai.ChatCompletion.create
    response = openai.Completion.create(
        engine="gpt-4o-mini",  # Replace 'model' with 'engine' for the new API
        prompt=text,
        max_tokens=100  # Adjust as needed for your responses
    )

    return response.choices[0].text.strip()

# Main chatbot function
def chatbot():
    print("🤖 Welcome to the AI Chatbot!")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break

        # Detect if input is Romanized Bangla
        if is_romanized_bangla_with_llm(user_input):
            print("🟢 Detected Romanized Bangla.")

            # Get LLM response in English
            response = get_chatbot_response(user_input)

            # Translate to Bangla
            translated_response = GoogleTranslator(source="en", target="bn").translate(response)

            print(f"Bot (বাংলা): {translated_response}")

        else:
            response = get_chatbot_response(user_input)
            print(f"Bot: {response}")

chatbot()

🤖 Welcome to the AI Chatbot!
You: ki khbr?


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
